# 3 - Bike Sharing Prediction

## Task description:

* **Training data**: whole 2011 and first 3 quarters of 2012.
* **Test data**: 4th quarter of 2012.  Do not fit your models with these data! They should just be used to see how good/bad your model predictions are.
* **Error metric**: R2 score (scikit-learn's default for regression).
* **Features to use**: at least the ones present in the data (except for cnt). Do not use both casual and registered columns, as cnt=casual+registered (you may use one, but not both). Additionally, you can use other sources of data you deem appropriate to predict from extra features.

## Exploratory Data Analysis (descriptive analytics) (4 points)

The first step of the analysis is to get familiar with the data. After importing the datasets we perform some analysis to have a general idea about the data. 
* **.head**: to understand the columns of the datasets
* **.describe**: to understand the distribution of the data 

In [1]:
import pandas as pd
import numpy as np

In order to predict the feature cnt we need to delete casual and registered (being these variable sub classes of cnt)

In [ ]:
!pip install kaggle

In [2]:
import zipfile
import requests 
import io 
import re

#!kaggle datasets list -s bike-sharing-dataset
!kaggle datasets download -d marklvl/bike-sharing-dataset --force

from io import BytesIO
zfname = 'bike-sharing-dataset.zip'

with zipfile.ZipFile(zfname) as zfile:
    for name in zfile.namelist():
        print(name)
        if re.search(r'\.zip$', name) is not None:
            # We have a zip within a zip
            zfiledata = BytesIO(zfile.read(name))
            with zipfile.ZipFile(zfiledata) as zfile2:
                day = pd.read_csv(zfile2.open("day.csv"))
                hour = pd.read_csv(zfile2.open("hour.csv"))



Bike-Sharing-Dataset.zip



  0%|          | 0.00/273k [00:00<?, ?B/s]
100%|##########| 273k/273k [00:00<00:00, 13.2MB/s]


In [ ]:
# importing the datasets
# day = pd.read_csv("day.csv")
# hour = pd.read_csv("hour.csv")
del hour["casual"]
del hour["registered"]

In [3]:
day.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [ ]:
hour.head()

In [ ]:
hour.describe()

In [ ]:
hour['cnt'].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [ ]:
# here we plot the hourly distribution of the bike ride count by weekdays, to understand daily usage patterns
plt.figure(figsize=(14, 4))
weekday_hr_gb = hour.groupby(["weekday", "hr"]).mean()["cnt"].reset_index(drop=False)
sns.pointplot(x="hr", y="cnt", hue="weekday", data=weekday_hr_gb)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
plt.title("hourly distribution, split by weekdays")
plt.show()

We notice the following: 
- weekdays - peaks are mornings and evenings
- weekends - peaks are during the day

In [ ]:
weathersit_gb = hour.groupby(["weathersit", "yr"]).mean()["cnt"].reset_index(drop=False)
weathersit_gb["yr"].replace([0, 1], [2011, 2012], inplace=True)
sns.pointplot(hue="yr", x="weathersit", y="cnt", data=weathersit_gb)
plt.title("avg hourly count of bikes for every weather situation")
plt.xticks(
    range(4),
    ["clear", "cloudy", "light \nprecipitation", "heavy \nprecipitation"],
    rotation=0,
    fontsize="12",
)
plt.show()

In [ ]:
hum_gb = hour.groupby(["hum", "yr"]).mean()["cnt"].reset_index(drop=False)
hum_gb["yr"].replace([0, 1], [2011, 2012], inplace=True)
sns.lineplot("hum", "cnt", hue="yr", data=hum_gb, palette="Set1_r")
# sb.lineplot('hum','cnt',data=hum_gb[hum_gb['yr']==2012])
plt.title("avg hourly count of bikes for humidity levels")
plt.legend([2011, 2012])
plt.show()

We have reasons to suspect correlation. The more humid it is, the less bikes are being rented out in 2011 the trend looks a bit different - where humidity is very low also less bikes are being used together with year it has some connection to our target value


In [ ]:
windspeed_gb = hour.groupby(["windspeed", "yr"]).mean()["cnt"].reset_index(drop=False)
windspeed_gb["yr"].replace([0, 1], [2011, 2012], inplace=True)
sns.lineplot("windspeed", "cnt", hue="yr", data=windspeed_gb, palette="Set1_r")
plt.title("avg hourly count of bikes for windspeed")
plt.legend([2011, 2012])
plt.show()

As expected, there is no significant trend here to be seen

In [ ]:
atemp_gb = hour.groupby(["atemp", "yr"]).mean()["cnt"].reset_index(drop=False)
atemp_gb["yr"].replace([0, 1], [2011, 2012], inplace=True)
sns.lineplot(hue="yr", x="atemp", y="cnt", data=atemp_gb, palette="Set1_r")
plt.title('avg hourly count of bikes for different "feels like" temperatures')
plt.legend([2011, 2012])
plt.show()

We see that the warmer it feels, the more bikes are being used but only up to a certain temperature. When it starts feeling very warm, less bikes are being rented out

In [ ]:
# here we will plot the count of rides per month, while also representing its distribution
ax = sns.boxplot(x="mnth", y="cnt", data=hour)

In [ ]:
# Here we plot the correlations among variables through a heatmap, which will be useful during the feature engineering phase.
corrMatt = hour[["temp", "atemp", "hum", "windspeed", "cnt"]].corr()

mask = np.array(corrMatt)
# Turning the lower-triangle of the array to false
mask[np.tril_indices_from(mask)] = False
fig, ax = plt.subplots()
sns.heatmap(corrMatt, mask=mask, vmax=0.8, square=True, annot=True, ax=ax)

plt.show()

## Data Cleaning
Here we star with cleaning the data. 
* **Step 1:** We split the variables into categorical and numerical to understand what variable can undergo what transformation
* **Step 2:** We check for NAs
* **Step 3:** We plot the different variables as boxplots to get a sense of the outlier distribution. 

### Features class 

In order to improve the quality of the predictions, it is possible to transform wrongly encoded numerical feature in categorical ones. In this case "hr", "weekday", "mnth", "season", "weathersit", "holiday", "workingday" are encoded as numerical feature but they are clearly categorical. 

In [ ]:
#here we split the variables into categorical and numerical under two different lists
from numpy import math
import numpy as np

catfeats = pd.DataFrame(hour.describe(include=["O"])).columns
numfeats = list(hour.select_dtypes(include=[np.number]).columns.values)

In [ ]:
print(catfeats)

In [ ]:
print(numfeats)

In [ ]:
#here we coerce the variable type to categorical for these features
categoryVariableList = [
    "hr",
    "weekday",
    "mnth",
    "season",
    "weathersit",
    "holiday",
    "workingday",
]
for var in categoryVariableList:
    hour[var] = hour[var].astype("category")

In [ ]:
hour.info()

### NAs

As it is possible to notice there are not NAs in the dataset. Therefore, no further action is required. 

In [ ]:
#here we go hunting for NAs
for column in hour:
    NAs = hour[column].isnull().sum()
    print(column + " " + str(NAs))

### Outliers 
Here we have a look on the outliers. We will however not exclude any, as after iterating, we understood that we get better results withouth doing that. 

In [ ]:
import seaborn as sn

In [ ]:
def show_outlier():
    fig, axes = plt.subplots(nrows=2, ncols=2)
    fig.set_size_inches(12, 10)
    sn.boxplot(data=hour, y="cnt", orient="v", ax=axes[0][0])
    sn.boxplot(data=hour, y="cnt", x="season", orient="v", ax=axes[0][1])
    sn.boxplot(data=hour, y="cnt", x="hr", orient="v", ax=axes[1][0])
    sn.boxplot(data=hour, y="cnt", x="workingday", orient="v", ax=axes[1][1])

    axes[0][0].set(ylabel="Count", title="Box Plot On Count")
    axes[0][1].set(xlabel="Season", ylabel="Count", title="Box Plot On Count Across Season")
    axes[1][0].set(
        xlabel="Hour Of The Day",
        ylabel="Count",
        title="Box Plot On Count Across Hour Of The Day",
    )
    axes[1][1].set(
        xlabel="Working Day", ylabel="Count", title="Box Plot On Count Across Working Day"
    )

In [ ]:
show_outlier()

## Feature Engineering
Here we switch the original columns, with some dummy ones, so to fit the format needs of our algorithms. 

In [ ]:
def generate_dummies(df, dummy_column):
    dummies = pd.get_dummies(df[dummy_column], prefix=dummy_column)
    df = pd.concat([df, dummies], axis=1)
    return df

In [ ]:
hour_m = pd.DataFrame.copy(hour)
dummy_columns = ["season", "yr", "mnth", "hr", "weekday", "weathersit"]
for dummy_column in dummy_columns:
    hour_m = generate_dummies(hour_m, dummy_column)

In [ ]:
# remove the original categorical variables: "season", "yr", "mnth", "hr", "weekday", "weathersit"
for dummy_column in dummy_columns:
    del hour_m[dummy_column]

In [ ]:
## drop also the variables 'instant' since it is irrelevant
del hour_m['instant']

In [ ]:
hour_m.head()

## Machine Learning (predictive analytics) (5 points)

In this part, we start applying the different models we want to apply. The order of this subchapter is the following:
* **Step 1: Train and test split**
* **Step 2: Regression model**
* **Step 3: Random forest**
* **Step 4: XGBoost**
* **Step 5: Combining models**
* **Step 6: Pipeline**


### Training test split

In [ ]:
# here we create the training set from the complete dataset we have, using the date as filtering item
X_train = hour_m.loc[hour_m["dteday"] < "2012-10-1"]
del X_train["cnt"]
del X_train["dteday"]
X_train.head()

In [ ]:
# here we create the test set from the complete dataset we have, using the date as filtering item
X_test = hour_m.loc[hour_m["dteday"] >= "2012-10-1"]
del X_test["cnt"]
del X_test["dteday"]
X_test.head()

In [ ]:
y_train = hour_m.loc[hour_m["dteday"] < "2012-10-1"]["cnt"]
y_train.head()

In [ ]:
y_test = hour_m.loc[hour_m["dteday"] >= "2012-10-1"]["cnt"]
y_test.head()

## Regression Model
Here we run our regression model, which is the first model we want to try as if with such a simple one we get good results, we would prefer it over more complex ones. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_rm = regr.predict(X_test)

In [ ]:
# here we calculate the r^2 for the model
r2_rm = r2_score(y_test, y_pred_rm)
r2_rm.round(2)

In [ ]:
accuracy_rm = cross_val_score(estimator=regr, X=X_train, y=y_train, cv=5)
accuracy_rm[1].round(2)

Lets compare the distribution of train and test results. the distribution of train and test looks very different. It confirms visually that our model has not predicted really in a proper way cnt.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(12, 5)
sn.distplot(y_test, ax=ax1, bins=50)
sn.distplot((y_pred_rm), ax=ax2, bins=50)

In [ ]:
plt.plot(y_test, y_test, "r--", y_test, y_pred_rm, "b,")
plt.show()

### Random forest
Here we run out random forest model. Also, using grid_search, we find the best parameters for this model. 

In [ ]:
# Grid Search
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
regressor = RandomForestRegressor()
parameters = [
    {"n_estimators": [150, 200, 250, 300], "max_features": ["auto", "sqrt", "log2"]}
]
grid_search = GridSearchCV(estimator=regressor, param_grid=parameters)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
best_parameters

In [ ]:
# Random Forest Regression model
# Use the best parameters found from above to build the model

regressor = RandomForestRegressor(n_estimators=200, max_features="auto")
regressor.fit(X_train, y_train)

# Predicting the values
y_pred_rf = regressor.predict(X_test)

In [ ]:
r2_rf = r2_score(y_test, y_pred_rf)
r2_rf.round(2)

In [ ]:
# Using k-fold cross validation to evaluate the performance of the model
accuracy_rf = cross_val_score(estimator=regressor, X=X_train, y=y_train, cv=5)
accuracy_rf = accuracy_rf.mean()
accuracy_rf.round(2)

Here we plot the variable importance to get a grasp of what the most important ones are, and the relative importance of each one as well.

In [ ]:
# Relative importance of features
feature_importance = regressor.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
plt.figure(figsize=(12, 10))
plt.barh(pos, feature_importance[sorted_idx], align="center")
plt.yticks(pos, X_train.columns[sorted_idx])
plt.xlabel("Relative Importance")
plt.title("Variable Importance")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(12, 5)
sn.distplot(y_test, ax=ax1, bins=50)
sn.distplot((y_pred_rf), ax=ax2, bins=50)

In [ ]:
plt.plot(y_test, y_test, 'r--', y_test, y_pred_rf, 'b,')
plt.show()

### XGBoost
Here we run the XGBoost model. 

In [ ]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [ ]:
# here we coerce the variable type to 'int' to let XGBoost run
X_train["holiday"] = X_train.holiday.astype("int")
X_train["workingday"] = X_train.workingday.astype("int")

In [ ]:
# here we coerce the variable type to 'int' to let XGBoost run
X_test["holiday"] = X_test.holiday.astype("int")
X_test["workingday"] = X_test.workingday.astype("int")

In [ ]:
xgb_model = xgb.XGBRegressor(
    objective="reg:linear",
    random_state=42,
    colsample_bytree=0.3,
    learning_rate=0.1,
    max_depth=5,
    alpha=10,
)
xgb_model.fit(X_train, y_train)
y_pred_xgb_standard = xgb_model.predict(X_test)
plt.scatter(y_test, y_pred_xgb_standard)
r2_xgb_standard = r2_score(y_test, y_pred_xgb_standard)

In [ ]:
accuracy_xgb_model = cross_val_score(estimator=xgb_model, X=X_train, y=y_train, cv=5)
accuracy_xgb_model = accuracy_xgb_model.mean()
accuracy_xgb_model.round(2)

In [ ]:
plt.plot(y_test, y_test, 'r--', y_test, y_pred_xgb_standard, 'b,')
plt.show()

In [ ]:
# tuning Hyperparameters
parameters_xgb = [
    {
        "n_estimators": [500, 1000],
        "max_depth": [4, 8, 12],
        "colsample_bytree": [0.4, 0.8],
    }
]
grid_search = GridSearchCV(estimator=xgb_model, param_grid=parameters_xgb)
grid_search = grid_search.fit(X_train, y_train)
best_parameters_xgb = grid_search.best_params_

In [ ]:
best_parameters_xgb

In [ ]:
# finetuned_model
model = xgb.XGBRegressor(max_depth=4, n_estimators=1000, colsample_bytree=0.4, seed=42)

model.fit(
    X_train,
    y_train,
    eval_metric="rmse",
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=True,
    early_stopping_rounds=10,
)

In [ ]:
# here we do the actual prediction using
y_pred_xgb = model.predict(X_test)

In [ ]:
r2_xgb = r2_score(y_test, y_pred_xgb)
r2_xgb.round(2)

In [ ]:
accuracy_xgb = cross_val_score(estimator=xgb_model, X=X_train, y=y_train, cv=5)
accuracy_xgb = accuracy_xgb.mean()
accuracy_xgb.round(2)

In [ ]:
# Relative importance of features
feature_importance = model.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + 0.5
plt.figure(figsize=(12, 10))
plt.barh(pos, feature_importance[sorted_idx], align="center")
plt.yticks(pos, X_train.columns[sorted_idx])
plt.xlabel("Relative Importance")
plt.title("Variable Importance")
plt.show()

In [ ]:
# here we plot to estimate the goodness of our prediction
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(12, 5)
sn.distplot(y_test, ax=ax1, bins=50)
sn.distplot((y_pred_xgb), ax=ax2, bins=50)

In [ ]:
plt.plot(y_test, y_test, 'r--', y_test, y_pred_xgb, 'b,')
plt.show()

## Combining Models
In this section we combine some models, to see if we manage to achieve better results than the ones of the model which alone achieves the best results. We use random forest, gradient boosting and extra trees. 

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from sklearn.metrics import r2_score

In order to make this happen given limited computational power, we resized the dataset. This code therefore serves to  improve the running time and testing the code quality rather than given actual predictions.

In [ ]:
X_train_sample = hour_m[0:688]  # one month as training data
del X_train_sample["cnt"]
del X_train_sample["dteday"]

X_test_sample = hour_m.loc[688:829]  # one week to be predicted
del X_test_sample["cnt"]
del X_test_sample["dteday"]

y_train_sample = hour_m[0:688]["cnt"]

y_test_sample = hour_m[688:830]["cnt"]

In [ ]:
# here we define the models
rf = ensemble.RandomForestClassifier()
gbm = ensemble.GradientBoostingClassifier()
et = ensemble.ExtraTreesClassifier()

combo = ensemble.VotingClassifier(
    estimators=[("rf", rf), ("gbm", gbm), ("et", et)],
    voting="soft",
    weights=[3, 5, 2],
    n_jobs=10,
)

combo.fit(X_train_sample, y_train_sample)

In [ ]:
#here we do the prediction
y_pred_combo = combo.predict(X_test_sample)

In [ ]:
# here we measure the r^2
r2_combo = r2_score(y_test_sample, y_pred_combo)
r2_combo.round(2)

In [ ]:
# here we measure the accuracy
accuracy_combo = cross_val_score(
    estimator=combo, X=X_train_sample, y=y_train_sample, cv=5
)
accuracy_combo = accuracy_combo.mean()
accuracy_combo.round(2)

In [ ]:
# here we plot our predictions' distribution vs. the actual distribution to estimate the goodness of the model.
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(12, 5)
sn.distplot(y_test_sample, ax=ax1, bins=50)
sn.distplot((y_pred_combo), ax=ax2, bins=50)

In [ ]:
plt.plot(y_test_sample, y_test_sample, 'r--', y_test_sample, y_pred_combo, 'b,')
plt.show()

## Pipeline
Here we try to build a pipeline, where we figure out our best parameters for the logistic regression we run after it. 

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
steps = [('scaler', StandardScaler()), ('SVM', SVC())]
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps) # define the pipeline object.

In [ ]:
parameteres = {'SVM__C':[0.001,0.1,10,100,10e5], 'SVM__gamma':[0.1,0.01]}

In [ ]:
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=5)
grid

In [ ]:
X_train_sample["holiday"] = X_train_sample.holiday.astype("int")
X_train_sample["workingday"] = X_train_sample.workingday.astype("int")
X_test_sample["holiday"] = X_test_sample.holiday.astype("int")
X_test_sample["workingday"] = X_test_sample.workingday.astype("int")

In [ ]:
grid.fit(X_train_sample, y_train_sample)
print (grid.score(X_test_sample,y_test_sample))
print (grid.best_params_)

In [ ]:
# Logistic Regression - Showing the Classification Report.
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(
    C=10,
    class_weight=None,
    dual=False,
    fit_intercept=True,
    intercept_scaling=1,
    max_iter=200,
    multi_class="ovr",
    n_jobs=1,
    penalty="l2",
    random_state=None,
    solver="liblinear",
    tol=0.0001,
    verbose=0,
    warm_start=False,
)
logmodel.fit(X_train_sample, y_train_sample)

# Predicting on Test
y_pred_pip_sample = logmodel.predict(X_test_sample)

In [ ]:
r2_pip = r2_score(y_test_sample, y_pred_pip_sample)
r2_pip.round(2)

In [ ]:
accuracy_pip = cross_val_score(estimator = logmodel, X = X_train_sample, y = y_train_sample, cv =5)
accuracy_pip = accuracy_pip.mean()
accuracy_pip.round(2)

Here we plot the distribution of the predictions and the actual distribution to see how similar they are. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(12, 5)
sn.distplot(y_test, ax=ax1, bins=50)
sn.distplot((y_pred_pip_sample), ax=ax2, bins=50)

In [ ]:
plt.plot(y_test_sample, y_test_sample, 'r--', y_test_sample, y_pred_pip_sample, 'b,')
plt.show()

## Wrap up
To sum up, here is once again the steps we went through: 
* **Step 1**: Exploratory data analysis: here we explore and get familiar with the data (graphically and numerically) 
    -  We plot the explanatory variables with the target variable to understand their relations graphically
* **Step 2**: Data cleaning: here we deal with outliers & NAs. 
   -  We change the class of certain variables to categorical to better fit our needs. Outliers did not need to be dropped as we get better results withouth this move. 
* **Step 3**: Feature engineering: we replace some of the columns with dummy columns in the right formats for our algorithms. 
* **Step 4**: Machine Learning: here we run different models individually, then we combine them and then we use a pipeline. 
    - Below a summary of the results of the various models

In [ ]:
#here we summarize in a table the metrics of the various models we run to provide an overview
data = [
    {
        "regression": r2_rm,
        "random_forest": r2_rf,
        "xgb_standard": r2_xgb_standard,
        "xgb_tuned": r2_xgb,
        "combo*": r2_combo,
        "pipeline*": r2_pip,
    },
    {
        "regression": accuracy_rm[1],
        "random_forest": accuracy_rf,
        "xgb_standard": accuracy_xgb_model,
        "xgb_tuned": accuracy_xgb,
        "combo*": accuracy_combo,
        "pipeline*": accuracy_pip,
    },
]

models_comparison = pd.DataFrame(
    data,
    index=["r2", "accuracy"],
    columns=[
        "regression",
        "random_forest",
        "xgb_standard",
        "xgb_tuned",
        "combo*",
        "pipeline*",
    ],
)

In [ ]:
models_comparison.round(2)

In [ ]:
# here we plot the predictions of the various models against the actual values, so to compare them again.

def comparison_show():
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

    ax1 = axs[0]
    index = ["regression", "random_forest", "xgb_tuned"]
    r2 = models_comparison.loc[["r2"], ["regression", "random_forest", "xgb_tuned"]].values[
        0
    ]
    bar_width = 0.35
    opacity = 0.8
    ax1.bar(index, r2, bar_width, alpha=opacity, color="b", label="r2")

    ax1.set_title("Scores by model")
    ax1.legend()

    ax2 = axs[1]
    index = ["regression", "random_forest", "xgb_tuned"]
    accuracy = models_comparison.loc[
        ["accuracy"], ["regression", "random_forest", "xgb_tuned"]
    ].values[0]
    bar_width = 0.35
    opacity = 0.8
    ax2.bar(index, accuracy, bar_width, alpha=opacity, color="r", label="accuracy")

    ax2.set_title("Scores by model")
    ax2.legend()

    plt.show()

In [ ]:
comparison_show()

As we can see, XGBoost tuned presents the best metrics by far. 

In [ ]:
# here we plot the predictions of the various models against the actual values, so to compare them again.

def comparison_show_plots():

    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(27, 22))

    ax1 = axs[0, 0]
    ax1.plot(y_test, y_test, "r--", y_test, y_pred_rm, "b,")
    ax1.set_title("Regression Model")

    ax2 = axs[0, 1]
    ax2.plot(y_test, y_test, "r--", y_test, y_pred_rf, "b,")
    ax2.set_title("Random Forest")

    ax1 = axs[1, 0]
    ax1.plot(y_test, y_test, "r--", y_test, y_pred_xgb_standard, "b,")
    ax1.set_title("XG Boost Standard")

    ax2 = axs[1, 1]
    ax2.plot(y_test, y_test, "r--", y_test, y_pred_xgb, "b,")
    ax2.set_title("XG Boost Tuned")


    plt.show()

In [ ]:
comparison_show_plots()

As we can clearly see, XGBoost after tuning reaches the best results. Compared to the other models, the performance is sensibly higher. The increase in the R^2 of the XGBoost, through hyperparameter tuning, was the turning point which allowed us to reach predictions which are this good. 